In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import tf_encrypted as tfe


/anaconda3/envs/py35_int64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
num_classes = 10
input_shape = (1, 784)

In [3]:
model = Sequential()

model.add(Dense(64,batch_input_shape=input_shape))
model.add(Activation('relu'))
model.add(Dense(num_classes, name='logit'))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

In [5]:
model.predict(np.ones([1,784]))

array([[  0.4851077 , -15.035619  ,   9.871048  ,  14.029106  ,
        -21.57302   ,   7.5741096 ,  -0.19556288,  -5.472397  ,
          4.8088765 , -10.46627   ]], dtype=float32)

In [6]:
from collections import OrderedDict

players = OrderedDict([
    ('server0', 'localhost:4000'),
    ('server1', 'localhost:4001'),
    ('server2', 'localhost:4002'),
])

config = tfe.RemoteConfig(players)
config.save('/tmp/tfe.config')

In [7]:
tfe.set_config(config)
tfe.set_protocol(tfe.protocol.SecureNN())

In [8]:
tf.reset_default_graph()
with tfe.protocol.SecureNN():
    tfe_model = tfe.keras.models.clone_model(model)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



In [9]:
# Set up a new tfe.serving.QueueServer for the shared TFE model
q_input_shape = (1, 784)
q_output_shape = (1, 10)

server = tfe.serving.QueueServer(
    input_shape=q_input_shape, output_shape=q_output_shape, computation_fn=tfe_model
)

In [ ]:
sess = tfe_model._tfe_session

In [ ]:
global request_ix
request_ix = 1

def step_fn():
    global request_ix
    print("Served encrypted prediction {i} to client.".format(i=request_ix))
    request_ix += 1

server.run(
    sess,
    num_steps=5,
    step_fn=step_fn)

Served encrypted prediction 1 to client.
